In [1]:
import pandas as pd
import re
import string
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tokenizers import Tokenizer
import pickle

In [2]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/soumyekumar/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/soumyekumar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/soumyekumar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
og_data = pd.read_csv("data/output_chunk_7.csv")

In [27]:
og_data_98 = og_data.sample(frac = 0.98) # work data for assignment
og_data_02 = og_data.drop(og_data_98.index) # test data to be used by prof
#og_data_02.to_csv('data/og_data_02.csv', index=False)  # already created csv file

In [5]:
og_data_98.head()

,category,text
5404,science,june 2021 portoro is hosting the 8th european ...
6441,unrest,israel has launched an air strike against gaza...
3364,labour,that the covid 19 pandemic has left organisati...
6978,weather,tropical storm danny makes south carolina coas...
7175,weather,weather inshore until 6 pm on wednesday will b...


In [6]:
og_data_02.count() # check if 2% is kept aside

category    144
text        144
dtype: int64

In [7]:
def clean_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = BeautifulSoup(text, "html.parser").get_text()  # Remove HTML tags
    # call stop words and remove them
    stop_words = stopwords.words('english') 
    removed_stopwords_text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    # Perform stemming
    stemmer = nltk.SnowballStemmer("english")
    return ' '.join(stemmer.stem(word) for word in removed_stopwords_text.split(' '))


og_data_98["clean_text"] = og_data_98["text"].apply(clean_text)
og_data_98.head()



,category,text,clean_text
5404,science,june 2021 portoro is hosting the 8th european ...,june portoro host th european congress mathem...
6441,unrest,israel has launched an air strike against gaza...,israel launch air strike gaza renew clash spar...
3364,labour,that the covid 19 pandemic has left organisati...,covid pandem left organis struggl secur manag...
6978,weather,tropical storm danny makes south carolina coas...,tropic storm danni make south carolina coastal...
7175,weather,weather inshore until 6 pm on wednesday will b...,weather inshor pm wednesday hazi misti foggi ...


In [15]:
from sklearn.model_selection import train_test_split

X_nb= og_data_98["clean_text"]
y_nb= og_data_98["category"]

X_train,X_test,y_train,y_test= train_test_split(X_nb,y_nb,test_size=.2,random_state=42)

# Multinomial Naive Bayes with Bag of Words
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score,classification_report

model = make_pipeline(CountVectorizer(), MultinomialNB())

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"MultinomialNB with Bag of Words accuracy: {accuracy:.3f}")

MultinomialNB with Bag of Words accuracy: 0.678


In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import numpy as np

cv_scores = cross_val_score(model, X_nb, y_nb, cv=StratifiedKFold(n_splits=3, shuffle=True), scoring='accuracy')

print(f"Cross-Validation Scores:{cv_scores}")

print(f"Mean Accuracy: {np.mean(cv_scores):.2f}")

Cross-Validation Scores:[0.66028912 0.66539116 0.6619898 ]
Mean Accuracy: 0.66


In [17]:
import tensorflow as tf
import transformers

In [18]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder

# Encode labels
le = LabelEncoder()
y_logistic = le.fit_transform(og_data_98['category'])
X_logistic = og_data_98['clean_text']

# Create pipeline
model2 = make_pipeline(
    TfidfVectorizer(max_features=5000),
    LogisticRegression(max_iter=1000)
)

# Train
model2.fit(X_logistic, y_logistic)

# Evaluate
print("Model 2 Accuracy:", model2.score(X_logistic, y_logistic))

Model 2 Accuracy: 0.9040532879818595


In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(og_data_98['clean_text'])
tokenizer.num_words = 5000
sequences = tokenizer.texts_to_sequences(og_data_98['clean_text'])
X_lstm = pad_sequences(sequences, maxlen=200)
y_lstm= le.fit_transform(og_data_98['category'])


# Build model
model3 = tf.keras.Sequential([
    tf.keras.layers.Embedding(5000, 128),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')
])

model3.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train
history = model3.fit(
    X_lstm, y_lstm,
    batch_size=32,
    epochs=20,
    validation_split=0.2
)

# Evaluate
print("Model 3 Validation Accuracy:", max(history.history['val_accuracy']))

Epoch 1/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 15s 77ms/step - accuracy: 0.0873 - loss: 2.8339 - val_accuracy: 0.1841 - val_loss: 2.4538
Epoch 2/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 18s 101ms/step - accuracy: 0.2376 - loss: 2.3102 - val_accuracy: 0.3102 - val_loss: 2.2181
Epoch 3/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.4133 - loss: 1.8017 - val_accuracy: 0.4200 - val_loss: 1.9613
Epoch 4/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 19s 108ms/step - accuracy: 0.6544 - loss: 1.1648 - val_accuracy: 0.5127 - val_loss: 1.8607
Epoch 5/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.7504 - loss: 0.8245 - val_accuracy: 0.5170 - val_loss: 1.9243
Epoch 6/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 19s 106ms/step - accuracy: 0.8581 - loss: 0.5078 - val_accuracy: 0.5078 - val_loss: 2.1642
Epoch 7/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.9038 - loss: 0.3559 - val_accuracy: 0.5595 - val_loss: 2.1388
Epoch 8/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 18s 104ms/step - accuracy: 0.9477 - loss: 0.

In [25]:

import joblib
from tensorflow.keras.models import save_model,load_model 


# Model 1: Original Model (Naive Bayes)
joblib.dump(model, 'NaiveBayes.pkl')

# Model 2: Logistic Regression
joblib.dump(model2, 'logreg_model.pkl')

# Model 3: LSTM
model3.save('lstm_model.h5')


# Loading function for all models
def load_all_models():
    model1 = joblib.load('original_model.pkl')
    model2 = joblib.load('logreg_model.pkl')
    model3 = load_model('lstm_model.h5')
    
    return model1, model2, model3

# Prediction function for all models
def predict_all_models(text, model1, model2, model3):
    cleaned = clean_text(text)
    
    # Model 1 (Naive Bayes)
    pred1 = model1.predict([cleaned])[0]
    
    # Model 2 (Logistic Regression)
    pred2 = model2.predict([cleaned])[0]
    
    # Model 3 (LSTM)
    sequence = tokenizer.texts_to_sequences([cleaned])
    padded = pad_sequences(sequence, maxlen=200)
    pred3 = model3.predict(padded).argmax(axis=1)[0]
    
    return {
        'NaiveBayes': le.inverse_transform([pred1])[0],
        'Logistic Regression': le.inverse_transform([pred2])[0],
        'LSTM': le.inverse_transform([pred3])[0]
    }


In [26]:
# Load models once at startup
model1, model2, model3= load_all_models()

# Make predictions
sample_text = "Archaeologists discover ancient temple ruins under city center"
predictions = predict_all_models(sample_text, model1, model2, model3)

print("Predictions:", predictions)


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.4.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.4.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MultinomialNB from version 1.4.2 when using version 1.5.1. This might lead to bre

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
Predictions: {'NaiveBayes': 'arts', 'Logistic Regression': 'arts', 'LSTM': 'labour'}
